In [1]:
# Load libraries
import pymongo
import json
import xmltodict
import os
from pathlib import Path
import csv
import pandas as pd

In [2]:
def recursive_clean_function(dict_to_clean):
    """
    Function that deletes redundant words from the name of the fields of the dictionaries contained within the main one.
    This recursive function will be used in order to modify each dictionary located below the current level.
    This function does not return anything, since it modifies the original dictionary directly.
    Args:
        data_dict: information from a researcher stored in a python dictionary
    """
    # Iterate through all keys of the current dictionary
    for key_one in list(dict_to_clean):
        key_one_clean = key_one
        
        # Same cleaning process as described in clean_dict_names function
        if key_one_clean.count(':') == 1:
            key_one_clean = key_one.split(':')[1]
            dict_to_clean[key_one_clean] = dict_to_clean[key_one]
            del dict_to_clean[key_one]
        # If the current key starts with @ it is metadata that may not be relevant to query
        # and may add noise to the json that will be added to the DB, so it is deleted
        if key_one_clean[0]=='@':
            del dict_to_clean[key_one_clean]
            continue
            
        # If the fields (value associated with the current key key_one_clean) are stored in a dictionary
        if isinstance(dict_to_clean[key_one_clean], dict):
            # If the current field is a dictionary, and all the fields contained within start with "@" (metadata),
            # it will be removed from the main dictionary in order to reduce noise
            # If not, then the recursive function will be called again for the dictionary associated with key_one_clean
            if all(dict_key.startswith('@') for dict_key in list(dict_to_clean[key_one_clean])):
                del dict_to_clean[key_one_clean]
                continue
            else:
                recursive_clean_function(dict_to_clean[key_one_clean])

        # It may happen that the fields are stored in a list,
        # therefore it is required to fetch the information contained within and create a new dictionary
        # To do so, the keys will be the positions of the elements in the list
        if isinstance(dict_to_clean[key_one_clean], list):
            list_to_dict_data = {str(i):dict_to_clean[key_one_clean][i] 
                                 for i in range(len(dict_to_clean[key_one_clean]))}
            dict_to_clean.update({key_one_clean:list_to_dict_data})
            recursive_clean_function(dict_to_clean[key_one_clean])

In [3]:
def clean_dict_names(data_dict):
    """
    Function that deletes redundant words from the name of the fields in order to make the future queries easy to do.
    If the input is a nested dictionary, then a recursive function will be used in order to modify each dictionary
    located below the current level.
    This function does not return anything, since it modifies the original dictionary directly.
    Args:
        data_dict: information from a researcher stored in a python dictionary
    """
    for key_zero in list(data_dict):
        # Knowing that the keys always can contain one colon, if there is one then it may follow
        # the format 'person:person'. The first word is used by all the fields in the same level (excluding the ones
        # starting with'@'),
        # and the second one describes the field of interest. The last one should be maintained in order to find the fields
        # by doing easier querys
        key_zero_clean = key_zero
        if key_zero_clean.count(':') == 1:
            key_zero_clean = key_zero.split(':')[1]
            data_dict[key_zero_clean] = data_dict[key_zero]
            del data_dict[key_zero]
        
        # If data_dict is a nested dictionary, then recursion will be applied
        if isinstance(data_dict[key_zero_clean], dict):
            recursive_clean_function(data_dict[key_zero_clean])

In [4]:
def extract_information_from_xml(xml_dict):
    """
    Function that fetches the relevant information from the .xml file
    and returns a dictionary ready to be uploaded to the database as a document.
    If the xml file is not correct (i.e. wrong dictionary keys) an empty dictionary will be delivered.
    Args:
        xml_dict: information from a researcher stored in a dictionary
    Returns:
        researcher_data: dictionary that contains only the desired information 
        from the researcher if and only if the xml format is correct. If not, this dictionary will remain empty
    """
    researcher_data = {}
    # First ensure that the main key is 'record:record' in order to avoid reading wrong files
    if 'record:record' in xml_dict:
        # Now verify that the relevant keys are contained in the dictionary
         if all(key in xml_dict['record:record'] for key in ('person:person', 'activities:activities-summary')):
            # Extract the values for both keys and store it in a new dictionary that will contain
            # only the relevant information, and therefore avoiding noise
            researcher_data = {k: xml_dict['record:record'][k] for k in ('person:person', 'activities:activities-summary')}
            
            # In order to easily locate the document in the future (if the document needs to be updated or deleted for instance),
            # a unique id will be defined. Knowing that each person has its unique id, it will be used as the document identification,
            # since we want one document per person
            researcher_data['_id']=researcher_data['person:person']['@path'].split('/')[1]
            
            # Before adding the dictionary to the database, a function will be applied
            # in order to clean the names of the fields. This step will make the queries easier to write
            clean_dict_names(researcher_data)
                
    return researcher_data


In [5]:
def open_data_folders(data_folder_path, mongodb_collection):
    """
    Function that iterates over all the folders located in the given path and opens the .xml files within.
    Args:
        data_folder_path: folder that contains all the directories with the .xml
        mongodb_collection: location within the database where the documents will be stored    
    """
    # Iterate over each folder that contains .xml files
    
    task5 = []
    for folder in os.listdir(data_folder_path):
        print(folder)
        # Iterate over each file and verify it has .xml format
        for file in os.listdir("".join([data_folder_path,"/",folder])):
            if not file.endswith('.xml'): 
                continue
            else:
                file_path = Path("".join([data_folder_path,"/",folder, "/", file]))
                with open(file_path, encoding='utf8') as xml_file:
                    # Transform the xml file to json, so the db input will be a dictionary
                    data_dict = xmltodict.parse(xml_file.read())
                    researcher_data = extract_information_from_xml(data_dict)
                    # Verify that the dictionary is not empty
                    if researcher_data:
                        researcher_relevant_information = {}
                        if 'name' in researcher_data['person']:

                            if 'family-name' in researcher_data['person']['name']:
                                researcher_relevant_information['family_name'] = researcher_data['person']['name']['family-name']
                            if 'given-names' in researcher_data['person']['name']:
                                researcher_relevant_information['given_names'] = researcher_data['person']['name']['given-names']
                            if 'credit-name' in researcher_data['person']['name']:
                                researcher_relevant_information['credit_name'] = researcher_data['person']['name']['credit-name']

                        researcher_relevant_information['_id'] = researcher_data['_id']
                        
                        # Define a flag that will be added to the researcher file
                        # If he has worked in Spain at least one time then this flag will be set as True
                        has_studied_in_spain_flag = False
                        
                        if 'works' in researcher_data['activities-summary']:
                            # Verify if there is only one organization or there are more than one
                            # To do so, if there are many, then the main dictionary keys will be integers ('0', '1'...)
                            # If not, then the key 'last-modified-date' will be there instead of the integers
                            if 'last-modified-date' not in researcher_data['activities-summary']['works']['group']:
                                for group in researcher_data['activities-summary']['works']['group']:
                                    group_info = researcher_data['activities-summary']['works']['group'][str(group)]

                                    researcher_position_data = {}
                                    if 'title' in group_info['work-summary']:
                                        researcher_position_data['title'] = group_info['work-summary']['title']['title']
                                    if 'type' in group_info['work-summary']:
                                        researcher_position_data['type'] = group_info['work-summary']['type']
                                    if 'publication-date' in group_info['work-summary']:
                                        if 'year' in group_info['work-summary']['publication-date']:
                                            researcher_position_data['year'] = group_info['work-summary']['publication-date']['year']
                                        if 'month' in group_info['work-summary']['publication-date']:
                                            researcher_position_data['month'] = group_info['work-summary']['publication-date']['month']
                                    if 'journal-title' in group_info['work-summary']:
                                        researcher_position_data['journal_title'] = group_info['work-summary']['journal-title']
                            else:
                                group_info = researcher_data['activities-summary']['works']['group']
                                researcher_position_data = {}
                                if 'title' in group_info['work-summary']:
                                    researcher_position_data['title'] = group_info['work-summary']['title']['title']
                                if 'type' in group_info['work-summary']:
                                    researcher_position_data['type'] = group_info['work-summary']['type']
                                if 'publication-date' in group_info['work-summary']:
                                    if 'year' in group_info['work-summary']['publication-date']:
                                        researcher_position_data['year'] = group_info['work-summary']['publication-date']['year']
                                    if 'month' in group_info['work-summary']['publication-date']:
                                        researcher_position_data['month'] = group_info['work-summary']['publication-date']['month']
                                if 'journal-title' in group_info['work-summary']:
                                    researcher_position_data['journal_title'] = group_info['work-summary']['journal-title']

                                # In order to follow the nomenclature when there are more than one position,
                                # the unique position will be stored in a dictionary, using as key the number 0
                                researcher_relevant_information['0'] = researcher_position_data
                        # Assign the information to the researcher name
                    if researcher_relevant_information:
                        task5.append(researcher_relevant_information)
                        # x = mongodb_collection.insert_one(researcher_relevant_information)
                        researcher_relevant_information = {}
    researcher_keys = []
    for researcher_dict in task5:
        researcher_keys.extend(list(researcher_dict))
    researcher_keys = list(set(researcher_keys))
    # Sort the list taking into account the integer keys, since we want the list
    # to have first the id and the names instead of the int
    #researcher_keys = sorted(researcher_keys, key=lambda x: (x.isdigit(), x))
    
    chars=[]
    digits=[]
    for value in researcher_keys:
        if value.isdigit():
            digits.append(int(value))
        else:
            chars.append(value)
    chars.sort()
    digits.sort()
    # Sort the digits as integers and transform them again to str
    digits=[str(elem) for elem in digits]
    chars.extend(digits)
    
    researcher_keys = chars
    '''with open('task5.csv', 'w', newline='', encoding='utf-8-sig') as output_file:
        dict_writer = csv.DictWriter(output_file, researcher_keys,  delimiter = "/")
        dict_writer.writeheader()
        dict_writer.writerows(task5)'''
    df = pd.DataFrame(task5)
    
    df = df[chars]
    df.to_csv('tarea5.csv',encoding='utf-8-sig', index=False, sep="/", header = chars)

In [6]:
def create_researchers_list(mongodb_client, data_folder_path):
    """
    Main function that aims to create a list with information from researchers stored in .xml files.
    Args:
        data_folder_path: folder that contains all the directories with the .xml
    """
    researchers_client = pymongo.MongoClient(mongodb_client)
    researchers_db = researchers_client["mydatabase"]
    researchers_col = researchers_db["researchers_work_data"]
    open_data_folders(data_folder_path, researchers_col)

In [7]:
client="mongodb://localhost:27017/"
path="Directorios"

In [8]:
create_researchers_list(mongodb_client=client, data_folder_path=path)

000
001
002
00X
01X
02X
